# Overview of Data Variables and Sources

In [9]:
# Loading libraries
import requests
import pandas as pd

In [10]:
def query_acs(var_code, api_key):
    results = None
    return None

## US Census Variables

These variables were taken from the US Census API, specifically from the [American Community Survey](https://www.census.gov/data/developers/data-sets/acs-5year.html). Where possible all variables are collected at a census track level.

In [11]:
# List of variables taken from US CENSUS bureau
census_vars = {
    "total_pop": "B01001_001E", # Total population
    "white_no_hispanic": "B01001H_001E", # White Alone, not Hispanic or Latino
    "year_built": "B25035_001E", # Median Year structure was built
    "internet_access": "B28002_013E", # Number of households with No Internet Access
    "total_households": "B09019_003E", # Total number of households
    "labor_force_rate": "S2301_C02_001E", # Labor Force Participation Rate for population 16 years or older
    "pop_16_older": "S2301_C01_001E", # Population 16 years or older
    "less_than_hs": "S2301_C01_032E", # Population 25 to 64 years that are Less than high school graduate
    "insured": "S2701_C02_001E", # Insured population
    "senior_living_alone": "S1101_C01_013E", # Number of households - householder 65+ years old living alone
    "above_200_poverty": "S0501_C01_106E", # Total population at or above 200% poverty level
    "occupied_housing_units": "S0102_C01_090E", # Total number of housing units
    "owner_occupied": "S0102_C01_091E", # Number of owner occupied housing units
    "renter_occupied": "S0102_C01_092E", # Number of renter occupied housing units
    "disabled": "S0102_C01_044E" # Population with any disability
}

In [16]:
# Function that queries US Census Bureau for a specific variable and state
def query_acs(var_name, var_code, state_fips, api_key):
    
    # General format for ACS data retrieval for all census tracts in a state
    ACS_BASE_URL = 'https://api.census.gov/data/2019/acs/acs5?get=NAME,'
    ACS_SUBJECT_URL = 'https://api.census.gov/data/2019/acs/acs5/subject?get=NAME,'
    ACS_TRACTS = '&for=tract:*&in=state:'
    ACS_KEY = '&key=' + api_key
    
    # Check if this is a detailed table variable or subject table variable
    base_url = ""
    if (var_code[0] == "S"):
        base_url = ACS_SUBJECT_URL
    elif(var_code[0] == "B"):
        base_url = ACS_BASE_URL
    else:
        print(f"Error Unknown type of variable {var_code}")
        return None
    
    # Builds request for US Census API
    req_url = base_url + var_code + ACS_TRACTS + state_fips + ACS_KEY
    
    # Recieve and process response from US Census API
    resp = requests.get(req_url)
    if ((resp.status_code) != 200):
        print(f"Error response code {resp.status_code} for request url\n{req_url}")
        return None
    
    dat = resp.json()
    df = pd.DataFrame(dat[1:], columns=dat[0])
    df["GEOID"] = df['state'].astype(str) + df['county'].astype(str) + df['tract'].astype(str)
    
    # Rename variable with a name instead of code
    df = df.rename(columns={var_code: var_name})
    
    return df

In [18]:
# Get API Key
census_api_key = ""
f = open("api_keys/us_census_api_key.txt", "r")
census_api_key = f.read().rstrip("\n")
f.close()

query_acs("total_pop", census_vars["total_pop"], "01", census_api_key)

,NAME,total_pop,state,county,tract,GEOID
0,"Census Tract 11, Jefferson County, Alabama",4781,01,073,001100,01073001100
1,"Census Tract 14, Jefferson County, Alabama",1946,01,073,001400,01073001400
2,"Census Tract 20, Jefferson County, Alabama",4080,01,073,002000,01073002000
3,"Census Tract 38.02, Jefferson County, Alabama",5291,01,073,003802,01073003802
4,"Census Tract 40, Jefferson County, Alabama",2533,01,073,004000,01073004000
...,...,...,...,...,...,...
1176,"Census Tract 104, Lauderdale County, Alabama",3023,01,077,010400,01077010400
1177,"Census Tract 113, Lauderdale County, Alabama",2049,01,077,011300,01077011300
1178,"Census Tract 116.02, Lauderdale County, Alabama",4447,01,077,011602,01077011602
1179,"Census Tract 102, Lauderdale County, Alabama",2175,01,077,010200,01077010200
